In [1]:
!pip install sklearn-evaluation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 47 kB 4.0 MB/s 
     |████████████████████████████████| 1.5 MB 37.6 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 843 kB 50.0 MB/s 
     |████████████████████████████████| 1.6 MB 62.6 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [2]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 25.2 MB/s 


In [3]:
import numpy as np
import h5py
import tensorflow as tf
import pandas as pd
from pandas import DataFrame

# Data utilities
from pandas import read_csv
import matplotlib.pyplot as plt
# Data visualization
import plotly.express as px

# Machine learning
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import cross_val_score
from sklearn_evaluation import plot

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!wget https://raw.githubusercontent.com/33220311/Extremophiles/main/MergedAnnotations1024.csv
!wget https://github.com/33220311/Extremophiles/raw/main/thermophilic_embedding.h5

--2022-10-26 05:06:21--  https://raw.githubusercontent.com/33220311/Extremophiles/main/MergedAnnotations1024.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122008 (119K) [text/plain]
Saving to: ‘MergedAnnotations1024.csv’

MergedAnnotations10 100%[===================>] 119.15K  --.-KB/s    in 0.004s  

2022-10-26 05:06:21 (30.7 MB/s) - ‘MergedAnnotations1024.csv’ saved [122008/122008]

--2022-10-26 05:06:21--  https://github.com/33220311/Extremophiles/raw/main/thermophilic_embedding.h5
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/33220311/Extremophiles/main/thermophilic_embedding.h5 [fol

## Utility Function

In [7]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [8]:
# Utility function: Display model score(Loss & Accuracy) across all sets.

def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)
  
  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [9]:
def equal_error_rate(y_true, y_pred):
    n_imp = tf.count_nonzero(tf.equal(y_true, 0), dtype=tf.float32) + tf.constant(K.epsilon())
    n_gen = tf.count_nonzero(tf.equal(y_true, 1), dtype=tf.float32) + tf.constant(K.epsilon())

    scores_imp = tf.boolean_mask(y_pred, tf.equal(y_true, 0))
    scores_gen = tf.boolean_mask(y_pred, tf.equal(y_true, 1))

    loop_vars = (tf.constant(0.0), tf.constant(1.0), tf.constant(0.0))
    cond = lambda t, fpr, fnr: tf.greater_equal(fpr, fnr)
    body = lambda t, fpr, fnr: (
        t + 0.001,
        tf.divide(tf.count_nonzero(tf.greater_equal(scores_imp, t), dtype=tf.float32), n_imp),
        tf.divide(tf.count_nonzero(tf.less(scores_gen, t), dtype=tf.float32), n_gen)
    )
    t, fpr, fnr = tf.while_loop(cond, body, loop_vars, back_prop=False)
    eer = (fpr + fnr) / 2

    return eer

## Open embedding file

In [74]:
proteins = []

In [75]:
with h5py.File('thermophilic_embedding.h5', 'r') as f:
    for new_identifier in f.keys():
        proteins.append((new_identifier, np.array(f[new_identifier])))

In [76]:
#proteins

In [77]:
# Basic Protocol 3 — Step 5
annotations = read_csv('MergedAnnotations1024.csv')

In [78]:
annotations[:3]

,identifier,label,set
0,G1PDH_AERPE,1,train
1,PGMI_AERPE,1,train
2,OFOB1_AERPE,1,train


In [79]:
# Basic Protocol 3 — Step 6
train_set = annotations[annotations.set == "train"]
test_set = annotations[annotations.set == "test"]

In [80]:
print(f"The train set contains {len(train_set)} samples, and we will test on {len(test_set)} samples.")

The train set contains 5090 samples, and we will test on 1278 samples.


In [81]:
# Basic Protocol 3 — Step 7

training_embeddings = list()
training_identifiers = train_set.identifier.values
training_labels = train_set.label.values
print(len(training_identifiers))
print(len(training_labels))

5090
5090


In [82]:
train_set.identifier.values

array(['G1PDH_AERPE', 'PGMI_AERPE', 'OFOB1_AERPE', ..., 'SPRL_BACHD',
       'MURD_BACHD', 'FADA_YERPE'], dtype=object)

In [83]:
testing_embeddings = list()
testing_identifiers = test_set.identifier.values
testing_labels = test_set.label.values
print(len(testing_identifiers))
print(len(testing_labels))

1278
1278


In [84]:
seq = dict(proteins)
delete = list()

In [85]:
for identifier in training_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            training_embeddings.append(embedding)
        else:
          delete.append(identifier)

In [86]:
for identifier in testing_identifiers:
        if identifier in seq:
            embedding = seq[identifier]
            testing_embeddings.append(embedding)

In [87]:
np.where(training_identifiers=='CYSO_AERPE')

(array([], dtype=int64),)

In [88]:
#training_identifiers = np.delete(training_identifiers,0)
#training_labels = np.delete(training_labels,0)

In [89]:
# A sanity check: make sure that the numbers are equal!
assert(len(training_identifiers) == len(training_embeddings))
assert(len(testing_identifiers) == len(testing_embeddings))

In [90]:
len(training_embeddings)

5090

In [91]:
training_identifiers

array(['G1PDH_AERPE', 'PGMI_AERPE', 'OFOB1_AERPE', ..., 'SPRL_BACHD',
       'MURD_BACHD', 'FADA_YERPE'], dtype=object)

In [92]:
delete

[]

## Training

In [93]:
arr_train = np.array(training_embeddings)
nsample, nx, ny = arr_train.shape
train_dataset = arr_train.reshape((nsample, nx*ny))
train_dataset.shape

(5090, 1024)

In [94]:
arr_test = np.array(testing_embeddings)
nsample, nx, ny = arr_test.shape
test_dataset = arr_test.reshape((nsample, nx*ny))
test_dataset.shape

(1278, 1024)

### MLP with Grid Search

In [95]:
# Basic Protocol 3 — Step 8

multilayerperceptron = MLPClassifier(solver='lbfgs', random_state=10, max_iter=1000)

parameters = {
    'hidden_layer_sizes': [(30,), (1024,),(20,15)],
    'learning_rate_init': [0.001, 0.0001, 0.01],
    'solver':['lbfgs', 'adam','sgd'],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
# Basic Protocol 3 — Step 9

classifiers = GridSearchCV(multilayerperceptron, parameters, cv=3, scoring="accuracy")
history = classifiers.fit(train_dataset, training_labels)
classifier = classifiers.best_estimator_

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning:

Stochastic Optimizer: Maximum 

In [ ]:
DataFrame(classifiers.cv_results_)

In [ ]:
df =pd.DataFrame(classifiers.cv_results_)
new_path = '/content/test.xls'
writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/MLPTMergedUnique.xlsx')

In [ ]:
plot.grid_search(classifiers.cv_results_, change= 'hidden_layer_sizes', kind='bar')

In [ ]:
params = classifier.get_params()
params

In [ ]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

In [ ]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
f1_performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
  f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
sd_f1 = np.std(f1_performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)
print(sd_f1)

In [ ]:
print(classification_report(testing_labels, predicted_testing_labels))

In [ ]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

In [ ]:
cv_results = DataFrame(classifiers.cv_results_)

cv_results[['param_hidden_layer_sizes','split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [ ]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [ ]:
sn = TP / float(TP + FN)
print(sn)

In [ ]:
sp = TN / float(TN + FP)
print(sp)

In [ ]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

### LR

In [237]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

In [238]:
lr = LogisticRegression()
lr_history = lr.fit(train_dataset, training_labels)
lr.score(test_dataset,testing_labels)

0.8709677419354839

In [239]:
scores = cross_val_score(lr, train_dataset, training_labels, scoring= 'f1', cv=5)
scores  

array([0.90243902, 0.99069767, 0.9953271 , 0.98360656, 0.95942721])

In [240]:
scores = cross_validate(lr, train_dataset, training_labels, scoring= ['accuracy', 'precision', 'recall','f1'], cv=5)
scores 

{'fit_time': array([0.04694891, 0.05183554, 0.05167246, 0.0540185 , 0.05571127]),
 'score_time': array([0.00991344, 0.01068926, 0.00990677, 0.01678896, 0.01011682]),
 'test_accuracy': array([0.90990991, 0.99099099, 0.99548533, 0.98419865, 0.96162528]),
 'test_precision': array([0.95360825, 0.9953271 , 1.        , 0.99056604, 0.99014778]),
 'test_recall': array([0.85648148, 0.98611111, 0.99069767, 0.97674419, 0.93055556]),
 'test_f1': array([0.90243902, 0.99069767, 0.9953271 , 0.98360656, 0.95942721])}

In [241]:
predicted_testing_labels = lr.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

Our model has an accuracy of 0.87


In [242]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
f1_performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
  f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
sd_f1 = np.std(f1_performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)
print(sd_f1)

0.12167219088995515
0.05862095387393939
0.07960935618666065


In [243]:
print(classification_report(testing_labels, predicted_testing_labels))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        18
           1       0.85      0.85      0.85        13

    accuracy                           0.87        31
   macro avg       0.87      0.87      0.87        31
weighted avg       0.87      0.87      0.87        31



In [244]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

0.7350427350427351


In [245]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)


In [246]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [247]:
sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

0.8461538461538461
0.8888888888888888


In [248]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [249]:
#lr.save_weights('/result/LR.h5')
print(lr.coef_.shape)
lr_weights = lr.coef_
print(lr_weights)
print(np.max(lr_weights))
print(np.min(lr_weights))

(1, 1024)
[[ 0.51097512  0.29052844  0.60518619 ...  0.54199794  0.67045527
  -0.40212728]]
1.3851175769996082
-1.3311573390512756


### CNN with GridSearch

In [28]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier

In [29]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from time import time

In [30]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [31]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [32]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [33]:
n_epochs = 30 # 30 
n_epochs_cv = 10 # 10  # reduce number of epochs for cross validation for performance reason

n_cv = 3
validation_ratio = 0.10

In [34]:
def create_cnn_model(pool_type='max', conv_activation='sigmoid', optimizer='adam', dropout_rate=0.10):
    # create model
    model = Sequential()
    
    # first layer: convolution
    #model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1))) 
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu',input_shape=(n_features,1)))
    # second series of layers: convolution, pooling, and dropout
    model.add(Conv1D(32, kernel_size=3, activation=conv_activation))  
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))     
    
    # third series of layers: convolution, pooling, and dropout    
    model.add(Conv1D(64, kernel_size=3, activation=conv_activation))   # 32   
    if pool_type == 'max':
        model.add(MaxPooling1D(pool_size=2))
    if pool_type == 'average':
        model.add(AveragePooling1D(pool_size=2))
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate))     
      
    # fourth series
    model.add(Flatten())         
    model.add(Dense(64, activation='sigmoid')) # 64
    # add a dropout layer if rate is not null    
    if dropout_rate != 0:
        model.add(Dropout(rate=dropout_rate)) 
        
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile( 
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy'],
        )    
    return model

In [35]:
cnn = create_cnn_model()

In [36]:
cnn.compile(
  optimizer='adam',
  loss='binary_crossentropy',  
  metrics=['accuracy'],
)

In [37]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1022, 128)         512       
                                                                 
 conv1d_1 (Conv1D)           (None, 1020, 32)          12320     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 510, 32)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 510, 32)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 508, 64)           6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 254, 64)          0         
 1D)                                                    

In [38]:
# optimize model 
start = time()

# create model
model = KerasClassifier(build_fn=create_cnn_model, verbose=1)
# define parameters and values for grid search 
param_grid = {
    'pool_type': ['max', 'average'],
    'conv_activation': ['relu', 'tanh'],    
    'epochs': [10,30],
    'optimizer': ['adam','sgd'],
}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  """


In [39]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=n_cv)
grid_result = grid.fit(train_dataset, Y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
24 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-pack

Epoch 1/30
160/160 [==============================] - 11s 63ms/step - loss: 0.1830 - accuracy: 0.9185
Epoch 2/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0546 - accuracy: 0.9825
Epoch 3/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0435 - accuracy: 0.9847
Epoch 4/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0347 - accuracy: 0.9886
Epoch 5/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0268 - accuracy: 0.9914
Epoch 6/30
160/160 [==============================] - 10s 61ms/step - loss: 0.0211 - accuracy: 0.9921
Epoch 7/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0171 - accuracy: 0.9947
Epoch 8/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0135 - accuracy: 0.9963
Epoch 9/30
160/160 [==============================] - 10s 62ms/step - loss: 0.0120 - accuracy: 0.9963
Epoch 10/30
160/160 [==============================] - 10s 62ms/step - loss: 0.009

In [40]:
def display_cv_results(search_results):
    print('Best score = {:.4f} using {}'.format(search_results.best_score_, search_results.best_params_))
    means = search_results.cv_results_['mean_test_score']
    stds = search_results.cv_results_['std_test_score']
    params = search_results.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print('mean test accuracy +/- std = {:.4f} +/- {:.4f} with: {}'.format(mean, stdev, param))   

In [41]:
# summarize results
print('time for grid search = {:.0f} sec'.format(time()-start))
display_cv_results(grid_result)

time for grid search = 2482 sec
Best score = 0.9581 using {'conv_activation': 'tanh', 'epochs': 30, 'optimizer': 'adam', 'pool_type': 'max'}
mean test accuracy +/- std = 0.9568 +/- 0.0266 with: {'conv_activation': 'relu', 'epochs': 10, 'optimizer': 'adam', 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'relu', 'epochs': 10, 'optimizer': 'adam', 'pool_type': 'average'}
mean test accuracy +/- std = 0.2693 +/- 0.0995 with: {'conv_activation': 'relu', 'epochs': 10, 'optimizer': 'sgd', 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'relu', 'epochs': 10, 'optimizer': 'sgd', 'pool_type': 'average'}
mean test accuracy +/- std = 0.9578 +/- 0.0306 with: {'conv_activation': 'relu', 'epochs': 30, 'optimizer': 'adam', 'pool_type': 'max'}
mean test accuracy +/- std = nan +/- nan with: {'conv_activation': 'relu', 'epochs': 30, 'optimizer': 'adam', 'pool_type': 'average'}
mean test accuracy +/- std = 0.3459 +/- 0.1974 with

In [42]:
DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_conv_activation,param_epochs,param_optimizer,param_pool_type,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,361.570163,0.775565,4.501583,0.120529,relu,10,adam,max,"{'conv_activation': 'relu', 'epochs': 10, 'opt...",0.975250,0.975840,0.919222,0.956771,0.026552,4
1,0.104582,0.003982,0.000000,0.000000,relu,10,adam,average,"{'conv_activation': 'relu', 'epochs': 10, 'opt...",NaN,NaN,NaN,NaN,NaN,9
2,359.663370,1.463212,4.981668,0.280163,relu,10,sgd,max,"{'conv_activation': 'relu', 'epochs': 10, 'opt...",0.382440,0.285209,0.140330,0.269326,0.099477,7
3,0.045576,0.004137,0.000000,0.000000,relu,10,sgd,average,"{'conv_activation': 'relu', 'epochs': 10, 'opt...",NaN,NaN,NaN,NaN,NaN,10
4,1065.359592,0.934102,4.890307,0.560047,relu,30,adam,max,"{'conv_activation': 'relu', 'epochs': 30, 'opt...",0.977018,0.981732,0.914505,0.957752,0.030641,2
5,0.104894,0.030610,0.000000,0.000000,relu,30,adam,average,"{'conv_activation': 'relu', 'epochs': 30, 'opt...",NaN,NaN,NaN,NaN,NaN,11
6,1081.531662,16.555154,3.905478,0.613164,relu,30,sgd,max,"{'conv_activation': 'relu', 'epochs': 30, 'opt...",0.612257,0.285209,0.140330,0.345932,0.197390,6
7,0.119941,0.033043,0.000000,0.000000,relu,30,sgd,average,"{'conv_activation': 'relu', 'epochs': 30, 'opt...",NaN,NaN,NaN,NaN,NaN,12
8,364.649951,2.427784,5.445194,0.215430,tanh,10,adam,max,"{'conv_activation': 'tanh', 'epochs': 10, 'opt...",0.974072,0.985268,0.913325,0.957555,0.031607,3
9,0.208666,0.021287,0.000000,0.000000,tanh,10,adam,average,"{'conv_activation': 'tanh', 'epochs': 10, 'opt...",NaN,NaN,NaN,NaN,NaN,13


In [43]:
df =pd.DataFrame(grid.cv_results_)
#new_path = '/content/test.xls'
#writer = pd.ExcelWriter(new_path, engine='xlsxwriter')
df.to_excel('/content/drive/MyDrive/SOM/CNNMergedUnique.xlsx')

In [44]:
classifier = grid.best_estimator_

params = classifier.get_params()
params

{'verbose': 1,
 'conv_activation': 'tanh',
 'epochs': 30,
 'optimizer': 'adam',
 'pool_type': 'max',
 'build_fn': <function __main__.create_cnn_model(pool_type='max', conv_activation='sigmoid', optimizer='adam', dropout_rate=0.1)>}

In [45]:
# Basic Protocol 3 — Step 10

predicted_testing_labels = classifier.predict(test_dataset)
accuracy = accuracy_score(testing_labels, predicted_testing_labels)

print(f"Our model has an accuracy of {accuracy:.2}")

3/3 [==============================] - 0s 14ms/step
Our model has an accuracy of 0.9


In [46]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score, balanced_accuracy_score
import numpy as np
        
bootstrap_performances = list()
performances = list()
f1_performances = list()
Y = np.array(testing_labels) # convert list of groundtruths to numpy
Yhat = np.array(predicted_testing_labels) # same same for predictions
n_samples = len(Y) # get number of samples 
n_bootstrap = 1000 # number of bootstrap iterations
metric = matthews_corrcoef # the metric you want to compute
for i in range(n_bootstrap): # for each bootstrap draw
  subset = np.random.choice(n_samples, n_samples, replace=True) 
  # create a random subset of your predictions/targets with replacement (this line will only generate the indices for list elements and the line below will grab the random subset with replacement
  bootstrap_performances.append( matthews_corrcoef(testing_labels[subset], predicted_testing_labels[subset]) )
  performances.append(accuracy_score(testing_labels[subset], predicted_testing_labels[subset]))
  f1_performances.append(f1_score(testing_labels[subset], predicted_testing_labels[subset]))
std_dev = np.std(bootstrap_performances) # compute std deviation over the bootstrapped performances
sd = np.std(performances)
sd_f1= np.std(f1_performances)
#print("Standard error after 1k bootstrapping iterations: {:.5f}".format(std_dev))|
#print(performances)
print(std_dev)
print(sd)
print(sd_f1)

0.07223074917532217
0.035570135899392265
0.047826961201984645


In [47]:
print(classification_report(testing_labels, predicted_testing_labels))

              precision    recall  f1-score   support

           0       0.89      0.95      0.92        42
           1       0.93      0.83      0.88        30

    accuracy                           0.90        72
   macro avg       0.91      0.89      0.90        72
weighted avg       0.90      0.90      0.90        72



In [48]:
print(matthews_corrcoef(testing_labels, predicted_testing_labels))

0.8001322641986388


In [49]:
from pandas import DataFrame
cv_results = DataFrame(grid.cv_results_)

cv_results[['param_conv_activation','split0_test_score', 'split1_test_score', 'split2_test_score']]

,param_conv_activation,split0_test_score,split1_test_score,split2_test_score
0,relu,0.975250,0.975840,0.919222
1,relu,NaN,NaN,NaN
2,relu,0.382440,0.285209,0.140330
3,relu,NaN,NaN,NaN
4,relu,0.977018,0.981732,0.914505
5,relu,NaN,NaN,NaN
6,relu,0.612257,0.285209,0.140330
7,relu,NaN,NaN,NaN
8,tanh,0.974072,0.985268,0.913325
9,tanh,NaN,NaN,NaN


In [50]:
# Further metrics
from sklearn.metrics import confusion_matrix

# Data visualization
import plotly.express as px

In [51]:
classes = np.unique(testing_labels)

confusion_matrix_data = confusion_matrix(testing_labels, predicted_testing_labels, labels=classes)

In [52]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

In [53]:
sn = TP / float(TP + FN)
print(sn)

0.8333333333333334


In [54]:
sp = TN / float(TN + FP)
print(sp)

0.9523809523809523


In [55]:
confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

### MLP

In [ ]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
#seed_value= 100
#tf.random.set_seed(seed_value)
n_cols = X_train.shape[1]
n_cols

In [ ]:
model = Sequential()
model.add(Dense(30,activation='relu', input_shape=(n_cols,)))
#model.add(Dense(2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08), loss="mse", metrics="accuracy", )

In [ ]:
checkpointer = ModelCheckpoint(
    filepath='folder/.{epoch:03d}-{val_loss:.3f}.h5',
    verbose=1,
    save_best_only=True)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=150)

In [ ]:
history = model.fit(X_train, y_train,  
   epochs = 1000, 
   verbose = 0, 
   validation_data=(X_test,y_test), shuffle=True, validation_split = 0.1
  # callbacks = [es, checkpointer]
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

In [ ]:
display_model_score(model,
    [X_train, y_train],
    [X_test, y_test],
    [X_test, y_test],
    256)

In [ ]:
classes = np.unique(y_test)

confusion_matrix_data = confusion_matrix(y_test, y_pred, labels=classes)

confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
plot_history(history)

In [ ]:
model.save_weights('MLP.h5')

### CNN

In [ ]:
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Activation
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.layers import Flatten
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from time import time

In [ ]:
X_train, X_test= train_dataset, test_dataset
y_train, y_test = training_labels, testing_labels

In [ ]:
Y_train = np.reshape(y_train,(len(y_train),1)).astype(int)
Y_test = np.reshape(y_test,(len(y_test),1)).astype(int)

In [ ]:
n_timesteps, n_features, n_outputs =train_dataset.shape[0], train_dataset.shape[1], Y_train.shape[1]

In [ ]:
# Building the CNN Model
cnn_model = Sequential()      # initializing the Sequential nature for CNN model

In [ ]:
cnn_model.add(Conv1D(filters=128, kernel_size=1, 
activation='relu',input_shape=(n_features,1)))
#cnn_model.add(Conv1D(filters=128, kernel_size=1, activation='relu'))
#cnn_model.add(Dropout(0.5))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(250, activation='relu'))
cnn_model.add(Dense(n_outputs, activation='sigmoid'))

In [ ]:
cnn_model.compile(loss='binary_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
                  metrics=['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

In [ ]:
# fit network
history = cnn_model.fit(train_dataset, Y_train, 
                        validation_data=(test_dataset, Y_test),
                        callbacks=[es],
                        epochs=500, batch_size=256, verbose=1
                        )

In [ ]:
plot_history(history)

In [ ]:
display_model_score(cnn_model,
    [train_dataset, Y_train],
    [test_dataset, Y_test],
    [test_dataset, Y_test],
    256)

In [ ]:
y_pred = cnn_model.predict(test_dataset)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(Y_test, y_pred))

In [ ]:
print(matthews_corrcoef(Y_test, y_pred))

In [ ]:
classes = np.unique(Y_test)

confusion_matrix_data = confusion_matrix(Y_test, y_pred, labels=classes)

confusion_matrix_figure = px.imshow(
    confusion_matrix_data,
    labels=dict(x="True label", y="Predicted label", color="# of samples"),
    x=classes,
    y=classes,
    color_continuous_scale='Gray_r'
)
confusion_matrix_figure.show()

In [ ]:
TP = confusion_matrix_data[1,1]
TN = confusion_matrix_data[0,0]
FP = confusion_matrix_data[0,1]
FN = confusion_matrix_data[1,0]

print(TP,TN, FP, FN)

sn = TP / float(TP + FN)
print(sn)
sp = TN / float(TN + FP)
print(sp)

In [ ]:
cnn_model.save_weights('CNN.h5')

## Visualization


In [ ]:
embeddings =list(map(lambda x: x[1], proteins))

In [ ]:
p = np.array(embeddings)
type(p)
p.shape
p = p.reshape(2809,1024)
p.shape

In [ ]:
def compute_pca(X, n_components=2):
    """
    Input:
        X: of dimension (m,n) where each row corresponds to a word vector
        n_components: Number of components you want to keep.
    Output:
        X_reduced: data transformed in 2 dims/columns + regenerated original data
    pass in: data as 2D NumPy array
    """

    ### START CODE HERE ###
    # mean center the data
    X_demeaned = X - X.mean(axis=0)

    # calculate the covariance matrix
    covariance_matrix = np.cov(X_demeaned, rowvar=False)

    # calculate eigenvectors & eigenvalues of the covariance matrix
    eigen_vals, eigen_vecs = np.linalg.eigh(covariance_matrix)

    # sort eigenvalue in increasing order (get the indices from the sort)
    idx_sorted = np.argsort(eigen_vals)
    
    # reverse the order so that it's from highest to lowest.
    idx_sorted_decreasing = list(reversed(idx_sorted))

    # sort the eigen values by idx_sorted_decreasing
    eigen_vals_sorted = eigen_vals[idx_sorted_decreasing]

    # sort eigenvectors using the idx_sorted_decreasing indices
    eigen_vecs_sorted = eigen_vecs[:, idx_sorted_decreasing]

    # select the first n eigenvectors (n is desired dimension
    # of rescaled data array, or dims_rescaled_data)
    eigen_vecs_subset = eigen_vecs_sorted[:, :n_components]

    # transform the data by multiplying the transpose of the eigenvectors with the transpose of the de-meaned data
    # Then take the transpose of that product.
    X_reduced = np.dot(eigen_vecs_subset.T, X_demeaned.T).T

    ### END CODE HERE ###

    return X_reduced

In [ ]:
result = compute_pca(p, 2)
plt.scatter(result[:, 0], result[:, 1])
plt.figure(figsize = (20, 16))
for i, word in enumerate(annotations.identifier):
    plt.annotate(word, xy=(result[i, 0] - 0.0005, result[i, 1] + 0.001))

plt.show()

In [ ]:
proteins_reduced = compute_pca(p, n_components=5)
print(proteins_reduced)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=5, svd_solver='full')
pca.fit(p)
print(pca.explained_variance_ratio_)
print(pca.singular_values_)
print(pca.score(p))
print(pca.transform(p))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
def reduce_dim(weights, components = 3, method = 'tsne'):
    """Reduce dimensions of embeddings"""
    if method == 'tsne':
        return TSNE(components, metric = 'cosine').fit_transform(weights)
    elif method == 'umap':
        # Might want to try different parameters for UMAP
        return UMAP(n_components=components, metric = 'cosine', 
                    init = 'random', n_neighbors = 5).fit_transform(weights)

In [ ]:
protein_r = reduce_dim(p, components = 2, method = 'tsne')
protein_r.shape

In [ ]:
from IPython.core.interactiveshell import InteractiveShell

# Set shell to show all lines of output
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
InteractiveShell.ast_node_interactivity = 'last'

plt.figure(figsize = (20, 16))
plt.plot(protein_r[:, 0], protein_r[:, 1], 'r.')
plt.xlabel('TSNE 1'); plt.ylabel('TSNE 2'); plt.title('Protein Embeddings Visualized with TSNE');

In [ ]:
gen = ['non thermophilic', 'themophilic']
ints = annotations.label

plt.figure(figsize = (10, 8))

# Plot embedding
plt.scatter(protein_r[:, 0], protein_r[:, 1], c = ints)#, cmap = plt.cm.tab10)

# Add colorbar and appropriate labels
cbar = plt.colorbar()
cbar.set_ticks([])
for j, lab in enumerate(gen):
    cbar.ax.text(1, (2 * j + 1) / ((10) * 2), lab, ha='left', va='center')
cbar.ax.set_title('Label', loc = 'left')


plt.xlabel('TSNE 1'); plt.ylabel('TSNE 2'); plt.title('TSNE Visualization of Protein Embeddings');

### BLSTM

In [ ]:
from keras.layers import Embedding, Bidirectional, CuDNNLSTM, GlobalMaxPooling1D, LSTM
from keras.regularizers import l2

In [ ]:
x_input = Input(shape=(1024,))
emb = Embedding(20, 512, input_length=1024)(x_input)
bi_rnn = Bidirectional(CuDNNLSTM(256))(emb) #, kernel_regularizer=l2(0.0001), recurrent_regularizer=l2(0.0001), bias_regularizer=l2(0.0001)
#bi_rnn = LSTM(512)(emb)
#x = Dropout(0.2)(bi_rnn)

# sigmoid classifier
x_output = Dense(1, activation='sigmoid')(bi_rnn)

blstm = Model(inputs=x_input, outputs=x_output)

In [ ]:
blstm.summary()

In [ ]:
# Early Stopping
es = EarlyStopping(monitor='val_loss', patience=150, verbose=1)

In [ ]:
blstm.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )

In [ ]:
history = blstm.fit(
     X_train, y_train,
     epochs=100, batch_size=512,
     validation_data=(X_test, y_test),
     callbacks=[es]
     )

In [ ]:
plot_history(history)

In [ ]:
display_model_score(blstm,
    [X_train, y_train],
    [X_test, y_test],
    [X_test, y_test],
    256)

In [ ]:
y_pred = blstm.predict(X_test)

In [ ]:
y_pred = np.reshape(y_pred,(len(y_pred),)).astype(int)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
blstm.save_weights('BLSTM.h5')